In [1]:
# Load this point cloud /home/shashank/Documents/UniBonn/Sem2/MSR_P04/DataSets/custom_dataset/sequence1_bin/49897000.bin
import open3d as o3d
import numpy as np

pcd = o3d.io.read_point_cloud("/home/shashank/Documents/UniBonn/Sem2/MSR_P04/DataSets/custom_dataset/sequence2/667354000.pcd")
o3d.visualization.draw_geometries([pcd])

In [2]:


''' # Boundin box pose estimation
[ 2.3751512  -7.905664   -0.9961796   4.599491    1.6807262   1.5213563
  -0.02062713] # x,y,z,l,w,h,theta
'''
def compute_bounding_box_object(X, Y, Z, Length, Width, Height, Yaw):
    # Create the bounding box vertices in its local frame
    # vertices = np.array([
    #     [Length / 2, 0, Width / 2],
    #     [Length / 2, 0, -Width / 2],
    #     [-Length / 2, 0, -Width / 2],
    #     [-Length / 2, 0, Width / 2],
    #     [Length / 2, Height, Width / 2],
    #     [Length / 2, Height, -Width / 2],
    #     [-Length / 2, Height, -Width / 2],
    #     [-Length / 2, Height, Width / 2]
    # ])

    vertices = np.array([
        [Length / 2, Width / 2, -Height/2],
        [Length / 2, -Width / 2, -Height/2],
        [-Length / 2, -Width / 2, -Height/2],
        [-Length / 2, Width / 2, -Height/2],
        [Length / 2, Width / 2, Height/2],
        [Length / 2, -Width / 2, Height/2],
        [-Length / 2, -Width / 2, Height/2],
        [-Length / 2, Width / 2, Height/2]

    ])

    # Create the rotation matrix based on Yaw
    rotation_matrix = np.array([
        [np.cos(Yaw), 0, np.sin(Yaw)],
        [0, 1, 0],
        [-np.sin(Yaw), 0, np.cos(Yaw)]
    ])

    # Rotate and translate the bounding box to its final pose
    rotated_vertices = np.dot(vertices, rotation_matrix.T)
    translated_vertices = rotated_vertices + np.array([X, Y, Z])

    return o3d.geometry.OrientedBoundingBox.create_from_points(o3d.utility.Vector3dVector(translated_vertices))
coordinate_axis = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0, 0, 0])
bb_obj = compute_bounding_box_object(2.3751512, -7.905664, -0.9961796, 4.599491, 1.6807262, 1.5213563, -0.02062713)
cropped_pcd = pcd.crop(bb_obj)
# Visualize the point cloud
o3d.visualization.draw_geometries([coordinate_axis,cropped_pcd])

In [3]:


def compute_bounding_box_edges(X, Y, Z, Length, Width, Height,Yaw ):
    # Create the bounding box vertices in its local frame
    # vertices = np.array([
    #     [Length / 2, 0, Width / 2],
    #     [Length / 2, 0, -Width / 2],
    #     [-Length / 2, 0, -Width / 2],
    #     [-Length / 2, 0, Width / 2],
    #     [Length / 2, Height, Width / 2],
    #     [Length / 2, Height, -Width / 2],
    #     [-Length / 2, Height, -Width / 2],
    #     [-Length / 2, Height, Width / 2]
    # ])

    vertices = np.array([
        [Length / 2, Width / 2, -Height/2],
        [Length / 2, -Width / 2, -Height/2],
        [-Length / 2, -Width / 2, -Height/2],
        [-Length / 2, Width / 2, -Height/2],
        [Length / 2, Width / 2, Height/2],
        [Length / 2, -Width / 2, Height/2],
        [-Length / 2, -Width / 2, Height/2],
        [-Length / 2, Width / 2, Height/2]

    ])
    # Create the rotation matrix based on Yaw
    rotation_matrix = np.array([
        [np.cos(Yaw), 0, np.sin(Yaw)],
        [0, 1, 0],
        [-np.sin(Yaw), 0, np.cos(Yaw)]
    ])

    # Rotate and translate the bounding box to its final pose
    rotated_vertices = np.dot(vertices, rotation_matrix.T)
    translated_vertices = rotated_vertices + np.array([X, Y, Z])

        # Define the edges of the bounding box
    edges = [
        [0, 1], [1, 2], [2, 3], [3, 0],
        [4, 5], [5, 6], [6, 7], [7, 4],
        [0, 4], [1, 5], [2, 6], [3, 7]
    ]
    # Create a Homogenous transformation matrix
    T = np.eye(4)
    T[:3, :3] = rotation_matrix
    T[:3, 3] = np.array([X, Y, Z])

    return o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(translated_vertices),
        lines=o3d.utility.Vector2iVector(edges)
    ), T

In [4]:
_,T = compute_bounding_box_edges(2.3751512, -7.905664, -0.9961796, 4.599491, 1.6807262, 1.5213563, -0.02062713)
print(T)

[[ 0.99978727  0.         -0.02062567  2.3751512 ]
 [ 0.          1.          0.         -7.905664  ]
 [ 0.02062567  0.          0.99978727 -0.9961796 ]
 [ 0.          0.          0.          1.        ]]


In [7]:
# Transform the cropped point cloud to the object frame
import copy
canonical_car = copy.deepcopy(cropped_pcd)
canonical_car.transform(np.linalg.inv(T))
o3d.visualization.draw_geometries([canonical_car, coordinate_axis])

In [6]:
# Save Canonical Car PCD in sequence folder in a subfolder called "canonical_pcd"
import os
try:
    o3d.io.write_point_cloud("/home/shashank/Documents/UniBonn/Sem2/MSR_P04/DataSets/custom_dataset/canonical_car.pcd", canonical_car)
except:
    # Create output directory
    if not os.path.exists("/home/shashank/Documents/UniBonn/Sem2/MSR_P04/DataSets/custom_dataset/sequence2/canonical_pcd"):
        os.makedirs("/home/shashank/Documents/UniBonn/Sem2/MSR_P04/DataSets/custom_dataset/sequence2/canonical_pcd")
    o3d.io.write_point_cloud("/home/shashank/Documents/UniBonn/Sem2/MSR_P04/DataSets/custom_dataset/sequence2/canonical_pcd/canonical_car.pcd", canonical_car)
print("Saved canonical_car.pcd")


Saved canonical_car.pcd


In [ ]:
o3d.